# Data Set generation for detection of pilot contamination in multi user Massive MIMO Systems

Vamos fazer um esquema para gerar um dataset básico com a razão dos autovalores da matriz R, quantidade de antenas e tal pra depois tentar usar algum algoritmo de ML.

### Imports

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as sig
import sys
import pandas as pd
from joblib import Parallel, delayed
from time import time
from IPython.display import display, clear_output

sys.path.append("../Python/")
import comm

### Function for signal propagation

In [2]:
def simularPropagacaoSinal(qtdSimbolosPiloto, bitsPorSimbolo, qtdAntenas, qtdUsuarios, potenciaEspiao, snr, qtdEspioes=1):
    
    # SIMULANDO OS CANAIS DO USUARIO E DO ESPIAO
    Haut = np.sqrt(0.5)*(np.random.normal(0, 1, size=(qtdAntenas, qtdUsuarios)) + 1j*np.random.normal(0, 1, size=(qtdAntenas, qtdUsuarios)))
    g    = np.sqrt(0.5)*(np.random.normal(0, 1, size=(qtdAntenas, qtdEspioes)) + 1j*np.random.normal(0, 1, (qtdAntenas, qtdEspioes)))

    # SEQUENCIA PILOTO ALEATORIA + MODULACAO PARA TODOS OS USUARIOS:
    bitStream = np.random.choice([0, 1], qtdSimbolosPiloto*bitsPorSimbolo*qtdUsuarios)
    symb      = comm.qpskmodulator(bitStream) # QPSK Modulator
    xp        = symb.reshape(qtdUsuarios, qtdSimbolosPiloto)

    # ESPIAO ENTRANDO NA JOGADA (SE A POTENCIA DELE FOR 0 ELE NAO ENTRA NA JOGADA):
    xpe  = np.sqrt(potenciaEspiao)*xp[0, :] # xpe vai ser a sequencia piloto do primeiro usuario multiplicada pela raiz da potencia do espiao
    xptx = np.concatenate((xp, [xpe])) # xptx sera a matriz xp com uma linha a mais: xpe
    H    = np.concatenate((Haut, g), axis=1) # H vai ser Haut com uma COLUNA a mais, que vai ser g

    # TRANSMISSAO PELO CANAL
    Y = np.dot(H, xptx) # fading
    Y = comm.awgn(Y, SNR=snr) # ruido branco
    
    return Y

### Main Function

In [3]:
def main(qtdSimbolosAtual, qtdBitsPorSimbolo, qtdAntenasAtual, qtdUsuariosAtual, potenciaEspiaoAtual, snrAtual, qtdEspioes, caminhoCSV):
    
    # PROPAGACAO DO SINAL
    Y = simularPropagacaoSinal(qtdSimbolosAtual, qtdBitsPorSimbolo, qtdAntenasAtual, qtdUsuariosAtual, potenciaEspiaoAtual, snrAtual, qtdEspioes)

    # CALCULANDO A RAZAO DOS AUTOVALORES
    razaoAtual = comm.razaoMatrizR(Y, qtdSimbolosAtual, M=qtdAntenasAtual, SNR=snrAtual)

    # FAZENDO O TARGET DO DATASET
    ataquePresente = 0
    if potenciaEspiaoAtual > 0:
        ataquePresente = 1

    # SALVANDO NO CSV - mode='a' significa append
    linhaNovaCSV = pd.DataFrame([[
        qtdUsuariosAtual, 
        qtdAntenasAtual, 
        qtdSimbolosAtual, 
        snrAtual, 
        razaoAtual, 
        potenciaEspiaoAtual, 
        ataquePresente
    ]]).to_csv(caminhoCSV, mode='a', header=False, index=False)

### System Parameters

In [4]:
# FIXED
nJobs             = -1
qtdEspioes        = 1
qtdBitsPorSimbolo = 2
repetibilidade    = 100
potenciaUsuario   = 1

# VARIABLES
rangeQtdUsuarios    = np.array([1,2,3])
rangeQtdAntenas     = np.array([64, 128, 192, 256])
rangePotenciaEspiao = np.arange(0.0, 2.51, 0.25)
rangeQtdSimbolos    = np.arange(10, 101, 5)
rangeSNRs           = np.arange(-30, 51, 2.5)

# TESTES RAPIDOS - DEIXAR COMENTADOOOOOOO
# rangeQtdUsuarios    = np.array([1]) # TESTES RAPIDOS 
# rangeQtdAntenas     = np.array([64, 128]) # TESTES RAPIDOS 
# rangePotenciaEspiao = np.arange(0.0, 1.1, 0.5) # TESTES RAPIDOS 
# rangeQtdSimbolos    = np.arange(20, 101, 20) # TESTES RAPIDOS 
# rangeSNRs           = np.arange(-10, 20, 5) # TESTES RAPIDOS 

### Simulate Scenarios While Creating CSV metadata

In [5]:
# INICIANDO O CSV
caminhoCSV = "../CSV/dataset_"+str(time())+".csv"
colunas    = [
    "qtdUsuarios", 
    "qtdAntenas", 
    "qtdSimbolos", 
    "snr", 
    "razaoAutovalores", 
    "potenciaEspiao_TARGET", 
    "ataquePresente_TARGET"
]
dataframe   = pd.DataFrame(columns=colunas).to_csv(caminhoCSV, index=False)
del dataframe
del colunas

# ITERANDO TODAS AS COMBINACOES POSSIVEIS DE VARIAVEIS 
for qtdUsuariosAtual in rangeQtdUsuarios:
    for qtdAntenasAtual in rangeQtdAntenas:
        for potenciaEspiaoAtual in rangePotenciaEspiao:
            for qtdSimbolosAtual in rangeQtdSimbolos:
                for snrAtual in rangeSNRs:
                    
                    # VAMOS PARALELIZAR PQ NEM DEUS TEM A PACIENCIA NECESSARIA KKKKK
                    Parallel(n_jobs=nJobs, verbose=100)(delayed(main)(qtdSimbolosAtual, qtdBitsPorSimbolo, qtdAntenasAtual, qtdUsuariosAtual, potenciaEspiaoAtual, snrAtual, qtdEspioes, caminhoCSV) for repetibilidadeAtual in range(repetibilidade))

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-1)]: Done  11 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-1)]: Done  15 tasks      | elapsed:   

[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.3s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0163s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0475s.) Setting batch_size=4.
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:    0.1s
[Para

[Parallel(n_jobs=-1)]: Done  40 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done  44 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done  48 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done  52 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.1369s.) Setting batch_size=16.
[Parallel(n_jobs=-1)]: Done  65 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done  73 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done  74 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done  82 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done  83 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done  84 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done  92 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done  93 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done  95 out of 100 | ela

[Parallel(n_jobs=-1)]: Done  36 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done  40 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done  44 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done  48 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done  52 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.1584s.) Setting batch_size=16.
[Parallel(n_jobs=-1)]: Done  65 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done  73 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done  74 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done  82 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done  83 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done  91 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done  92 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done  93 tasks      | ela

[Parallel(n_jobs=-1)]: Done  82 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done  83 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done  84 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done  92 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done  93 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done  95 out of 100 | elapsed:    0.4s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  97 out of 100 | elapsed:    0.5s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.5s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0281s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done   5 tasks      | elaps

...

[Parallel(n_jobs=-1)]: Done  22 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0866s.) Setting batch_size=8.
[Parallel(n_jobs=-1)]: Done  32 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done  36 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done  40 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done  44 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done  48 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done  52 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.1389s.) Setting batch_size=16.
[Parallel(n_jobs=-1)]: Done  72 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done  73 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done

[Parallel(n_jobs=-1)]: Done  36 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done  40 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done  44 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done  48 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done  52 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done  72 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done  73 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done  81 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done  82 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done  83 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done  84 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done  85 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done  86 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done  87 out of 100 | elapsed:    0.5s remainin

[Parallel(n_jobs=-1)]: Done  52 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.1934s.) Setting batch_size=16.
[Parallel(n_jobs=-1)]: Done  65 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done  73 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done  74 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done  75 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done  83 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done  84 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done  92 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done  93 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done  95 out of 100 | elapsed:    0.5s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  97 out of 100 | elapsed:    0.6s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.6s finished
[

[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0906s.) Setting batch_size=4.
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done  20 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done  22 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done  28 tasks      | elap

KeyboardInterrupt: 